In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv(r'E:\deep_learning.csv')
data.head()

,Number of times pregnant,glucose concentration,blood pressure,Triceps thickness,insulin level,Body mass index,Diabetes pedigree function,Age,Diabetes result
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
data.shape

(768, 9)

In [4]:
data.describe()

,Number of times pregnant,glucose concentration,blood pressure,Triceps thickness,insulin level,Body mass index,Diabetes pedigree function,Age,Diabetes result
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
def replace_zero(df, field, target):
    mean_by_target = df.loc[df[field] != 0, [field, target]].groupby(target).mean()
    data.loc[(df[field] == 0)&(df[target] == 0), field] = mean_by_target.iloc[0][0]
    data.loc[(df[field] == 0)&(df[target] == 1), field] = mean_by_target.iloc[1][0]

In [6]:
# replacing zeroes
zero_cols = ['glucose concentration', 'blood pressure', 'Triceps thickness', 'insulin level', 'Body mass index']
for col in zero_cols:   
    replace_zero(data, col, 'Diabetes result') 

In [7]:
# split data

X = data.drop('Diabetes result', axis=1)
y = data['Diabetes result']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.size)
print(y_test.size)

(614, 8)
(154, 8)
614
154


In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
## Create our model
model = Sequential()

model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid' ))

In [10]:
# Compile the model                                         
model.compile(loss='binary_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [11]:
# Training
es_call = EarlyStopping(monitor='loss', patience=5)

hist = model.fit(X_train, y_train,
                 validation_data=(X_test, y_test),
                 epochs=100,
                 batch_size=8,
                 callbacks=[es_call])

Epoch 1/100
77/77 [==============================] - 0s 3ms/step - loss: 14.0542 - accuracy: 0.6287 - val_loss: 4.0575 - val_accuracy: 0.5130
Epoch 2/100
77/77 [==============================] - 0s 2ms/step - loss: 2.4191 - accuracy: 0.6270 - val_loss: 2.9166 - val_accuracy: 0.6039
Epoch 3/100
77/77 [==============================] - 0s 2ms/step - loss: 1.7805 - accuracy: 0.6840 - val_loss: 2.3216 - val_accuracy: 0.6558
Epoch 4/100
77/77 [==============================] - 0s 2ms/step - loss: 1.4337 - accuracy: 0.7068 - val_loss: 1.8528 - val_accuracy: 0.7013
Epoch 5/100
77/77 [==============================] - 0s 2ms/step - loss: 1.2624 - accuracy: 0.7248 - val_loss: 1.7026 - val_accuracy: 0.7013
Epoch 6/100
77/77 [==============================] - 0s 2ms/step - loss: 1.1544 - accuracy: 0.7150 - val_loss: 1.6698 - val_accuracy: 0.6623
Epoch 7/100
77/77 [==============================] - 0s 1ms/step - loss: 1.0919 - accuracy: 0.7313 - val_loss: 1.3999 - val_accuracy: 0.7143
Epoch 8/100


In [12]:
#hist.history

In [13]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [14]:
y_pred.size

154

In [15]:
from sklearn.metrics import precision_score
scores1 = precision_score(y_test,y_pred)
print('precision_score test: '+str(scores1))

precision_score test: 0.6785714285714286


In [16]:
from sklearn.metrics import recall_score
scores2 = recall_score(y_test,y_pred)
print('recall_score test: '+str(scores2))

recall_score test: 0.34545454545454546


In [17]:
from sklearn.metrics import confusion_matrix
print('confusion matrix for test data: \n'+ str(confusion_matrix(y_test,y_pred)))

confusion matrix for test data: 
[[90  9]
 [36 19]]


In [18]:
from sklearn.metrics import classification_report, roc_curve,roc_auc_score
print(classification_report(y_test,y_pred))
print('accuracy_score: '+ str(accuracy_score(y_test,y_pred)))

              precision    recall  f1-score   support

           0       0.71      0.91      0.80        99
           1       0.68      0.35      0.46        55

    accuracy                           0.71       154
   macro avg       0.70      0.63      0.63       154
weighted avg       0.70      0.71      0.68       154



NameError: name 'accuracy_score' is not defined

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_curve,roc_auc_score
from matplotlib import pyplot


# Compute predicted probabilities: y_pred_prob
y_pred_prob = model.predict_proba(X_test)

# Generate ROC curve values: fpr, tpr, thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

############ Plot ROC curve ############
import matplotlib.pyplot as plt
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
########################################
print('roc_auc_score: '+str(roc_auc_score(y_test,y_pred_prob)))

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print(f"Accuracy on test set: {scores[1]*100:.2f}%")

In [ ]:
# save the model to disk
import os
filename = r'finalized_model.sav'
model.save(filename)